# HW 2

#### Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler,QuantileTransformer
from scipy.spatial.distance import cdist
from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge

In [2]:
import Lowess
import Logistic as LowessLR
import g_boost

In [3]:
#!pip install xgboost

#### Kernels

In [3]:
# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))
    
# Tricubic Kernel
def Tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)
    
# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))
    
# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

## Part 1

Create your class that implements the Gradient Boosting concept, based on the locally weighted regression method (Lowess class), and that allows a user-prescribed number of boosting steps. The class you develop should have all the mainstream useful options, including “fit,” “is_fitted”, and “predict,” methods. Show applications with real data for regression, 10-fold cross-validations and compare the effect of different scalers, such as the “StandardScaler”, “MinMaxScaler”, and the “QuantileScaler”. In the case of the “Concrete” data set, determine a choice of hyperparameters that yield lower MSEs for your method when compared to the eXtream Gradient Boosting library.

In [4]:
#data = pd.read_csv('../data/concrete.csv')
data = pd.read_csv('./data/concrete.csv')

X = data.drop(columns='strength').values
y = data['strength'].values

In [5]:
def k_fold_cross_validation(model):
    results = {}

    # list of scalers to iterate through
    scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "QuantileScaler": QuantileTransformer(n_quantiles=min(100, X.shape[0]), output_distribution='normal')
    }

    # initializing kfold cross-validation
    kf = KFold(n_splits=10, shuffle=True)

    for scaler_name, scaler in scalers.items():
        # scaling features using the current scaler
        X_scaled = scaler.fit_transform(X)

        # making list to store mse for each fold
        mse_scores = []

   
        for train_index, test_index in kf.split(X_scaled):
            # splitting into testing and training data
            X_train, X_test = X_scaled[train_index], X_scaled[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # fitting model on train data
            model.fit(X_train, y_train)

            # predicing on test data
            y_pred = model.predict(X_test)
            
            # calculate and append mse
            mse = mean_squared_error(y_test, y_pred)
            mse_scores.append(mse)

        # calculate the mean of the mse for each scaler
        results[scaler_name] = np.mean(mse_scores)
    return results

#### Our own gradient boosting

In [10]:
lowess_model = Lowess.Lowess(kernel=Gaussian)
ridge_model = Lowess.Lowess(kernel=Gaussian)
booster = g_boost.GradientBooster(lowess_model, ridge_model, boosting_steps=3)  # calling booster class with combines two models and performing three boosting steps

In [12]:
# performing k fold validation using function above using booster model
results = k_fold_cross_validation(booster)
results

{'StandardScaler': 53.9334824408945,
 'MinMaxScaler': 253.39539809478447,
 'QuantileScaler': 31.218286982812863}

#### eXtream Gradient Boosting

In [9]:
xgboost_model = xgb.XGBRegressor()  # creating xg boost regressor model

In [ ]:
# performing k fold validation using function above using xg boost model
results_xgb = k_fold_cross_validation(xgboost_model)
results_xgb

### Key Findings

Our lowest MSE for our own gradient boosting locally weighted regression method was using the Quantile Scaler with an MSE of 29.11. We used 3 boosting steps meaning the model has gone through three iterations of adding new decision trees to improve its predictions. We weren't able to reduce the MSE of own method below the eXtreme Gradient Boosting model with had the lowest MSE of 16.60 using the Quantile Scaler (like our method). 

## Part 2
Implement your own version of Locally Weighted Logistic Regression and compare its performance on the Iris data set with the version presented in this article: https://calvintchi.github.io/classical_machine_learning/2020/08/16/lwlr.html.

### Importing iris data, TTS, and Scaling

In [11]:
# importing data from sklearn
iris = datasets.load_iris()

In [12]:
# defining x and y
X = iris.data
y = iris.target

In [13]:
# performing tts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
# scaling data using MinMax Scaler, can also use other scalers which are currently commented out
scaler = MinMaxScaler()
#scaler = StandardScaler()
#scaler = QuantileTransformer()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Model Training and Evaluation

In [15]:
# creating the model with Gaussian kernel
model = LowessLR.LocallyWeightedLogisticRegression(kernel=Gaussian, lr=0.01, tau=0.2)

#model = LowessLR.LocallyWeightedLogisticRegression(kernel=Tricubic, lr=0.01, tau=0.05) 
#model = LowessLR.LocallyWeightedLogisticRegression(kernel=Epanechnikov, lr=0.01, tau=0.05) 
#model = LowessLR.LocallyWeightedLogisticRegression(kernel=Quartic, lr=0.01, tau=0.05) 

In [16]:
# fitting model to training data
model.fit(X_train, y_train)

In [17]:
# using trained model to make predictions on test data
predictions = model.predict(X_test)

In [ ]:
# Calculating accuracy of predictions compared to the true labels (y_test) and printing results
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

### Comparing results to Calvin Chi Paper

In [19]:
# version presented in Calvin Chi article

class locally_weighted_logistic_regression(object):
    
    def __init__(self, tau, reg = 0.0001, threshold = 1e-6):
        self.reg = reg
        self.threshold = threshold
        self.tau = tau
        self.w = None
        self.theta = None
        self.x = None

    def weights(self, x_train, x):
        sq_diff = (x_train - x)**2
        norm_sq = sq_diff.sum(axis = 1)
        return np.ravel(np.exp(- norm_sq / (2 * self.tau**2)))

    def logistic(self, x_train):
        return np.ravel(1 / (1 + np.exp(-x_train.dot(self.theta))))

    def train(self, x_train, y_train, x):
        self.w = self.weights(x_train, x)
        self.theta = np.zeros(x_train.shape[1])
        self.x = x
        gradient = np.ones(x_train.shape[1]) * np.inf
        while np.linalg.norm(gradient) > self.threshold:
            # compute gradient
            h = self.logistic(x_train)
            gradient = x_train.T.dot(self.w * (np.ravel(y_train) - h)) - self.reg * self.theta
            # Compute Hessian
            D = np.diag(-(self.w * h * (1 - h)))
            H = x_train.T.dot(D).dot(x_train) - self.reg * np.identity(x_train.shape[1])
            # weight update
            self.theta = self.theta - np.linalg.inv(H).dot(gradient)
    
    def predict(self,x):  # adjusted slightly to allow for input feature
        return np.array(self.logistic(x) > 0.5).astype(int)

In [20]:
# training one v rest models
model_dict = {}  # dictionary to store models

# training a model for each class
for cls in np.unique(y_train):  # iterating over each unique class label in the training data
    binary_y_train = (y_train == cls).astype(int)  # creating binary target variable for current class
    model = locally_weighted_logistic_regression(tau=.02)  # initializing model
    model.train(X_train, binary_y_train, X_train)  # training  model
    model_dict[cls] = model  # storing model for given class

In [21]:
# making predictions
predictions = []  # initializing list to store predictions

for x_test in X_test: # iterating over each test sample
    class_probs = [] # initializing list to store probabilities for each class
    
    for cls, model in model_dict.items():  # iterating over the items in model_dict
        prob = model.predict(x_test.reshape(1, -1))  # calling predict method of current model to get prediction
        class_probs.append(prob[0])  # appends predicted probability for current class to class_probs list
    
    # after evaluating all classes for current test sample, determine class with highest probability
    predictions.append(np.argmax(class_probs))

In [ ]:
# Calculating accuracy of predictions compared to the true labels (y_test) and printing results

accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

### Key Findings

Our Locally Weighted Logistic Regression model tends to perform better on the iris dataset than the model Calvin Chi presented. The accuracy rates are both quite variable, but our model generally performs better. We did choose to use the One vs Many approach of classification with both models. Since in the iris dataset there are three classifications, three separate logistic regression models are created. So these three models would be:
1. Setosa vs Not setosa
2. Versicolour vs Not Versicolour
3. Virginia vs Not Virginia

Each model outputs a probability that that instance belongs in the class. The final class prediction is then determined to be the one with the highest probability of the three models. 